In [ ]:
import pandas as pd
import numpy as np

NLP file consists of text messages and what category do they belong to (Sports and Not Sports) 

In [ ]:
df=pd.read_csv("C:/Users/Aditya/Desktop/NLP.csv")

In [ ]:
df

,Text,Class
0,It was a good game,Sports
1,It was a well fought election,NS
2,What a match it was,Sports
3,The teams fought well,Sports
4,The election was,NS


In [ ]:
#creating training data (data sample/row 1,2,4)
train=df.head(2)
train=train.append(df.iloc[3])
train=train.reset_index(drop=True)
train

,Text,Class
0,It was a good game,Sports
1,It was a well fought election,NS
2,The teams fought well,Sports


In [ ]:
#creating test data (data sample/row 3,5)
Test=df.tail(1)
Test=Test.append(df.iloc[2])
Test=Test.reset_index(drop=True)
Test

,Text,Class
0,The election was,NS
1,What a match it was,Sports


####**Converting messages to numerical data -**

**Method 1 : Manually**

In [ ]:
#This code is used to get distinct words from all texts with help of sets and use it as column names for df2 :
col=set()

colval=[1]

for i in range(0,len(train['Text'])):
    
    #split each sentence into its words and store them as an array in list1
    string=train["Text"][i]
    list1=string.split(" ")
        
    for j in range(0,len(list1)):
        col.add(list1[j])

#This code is used to create train_num with all elements 0 and use set 'col' as column names for df2 :
col1=list(col)

train_num=pd.DataFrame(0, index=np.arange(0, train.shape[0]), columns=col1)

#We put values of each column in train_num as their frequency of occurence in that row number's text message
for i in range(0,len(train['Text'])):
    string=train["Text"][i]
    list1=string.split(" ")
    for j in range(0,len(list1)):
        train_num[list1[j]][i]=train_num[list1[j]][i]+1
        
train_num['Output']=train['Class']


**OR**

**Method 2 : Using CountVectorizer**

In [ ]:
#The countvectorizer will take each distinct word from each data sample and create a column for it
#The values in this column with correspond to their frequency of occurence in correpsonding data sample/row
from sklearn.feature_extraction.text import CountVectorizer

#this token pattern recognizes single letter words like "a", by default the countvectorizer doesn't recognize words less than 2 letters
cv=CountVectorizer(stop_words=None, token_pattern=r"(?u)\b\w+\b")
x=cv.fit_transform(train['Text'])
arr=x.toarray()
arr

array([[1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 1, 1, 0, 1]], dtype=int64)

In [ ]:
train_num=pd.DataFrame(arr, columns =cv.get_feature_names())
train_num['Output']=train['Class']
#train_num consists of each distinct word in the training data and its frequency of occurence in each corresponding row/sentence
train_num

,a,election,fought,game,good,it,teams,the,was,well,Output
0,1,0,0,1,1,1,0,0,1,0,Sports
1,1,1,1,0,0,1,0,0,1,1,NS
2,0,0,1,0,0,0,1,1,0,1,Sports


In [ ]:
train

,Text,Class
0,It was a good game,Sports
1,It was a well fought election,NS
2,The teams fought well,Sports


In [ ]:
Num=0

Dict1={}

#train_col contains all distinct values in input training data
train_col=train_num.columns
#train_out_col contains all distinct values of target variable/output coulumn of training data
train_out_col=train_num['Output'].value_counts().index

for i in range(0,len(train_out_col)):
    for j in range(0,(len(train_col)-1)):

        #Num is probability of a particular input value (word) and an output value occuring together in given data samples
        #add the frequency of attribute value for that row
        Num=train_num[train_num['Output']==train_out_col[i]][train_col[j]].sum()

        #Probability of a particular output value occuring
        denom=train_num[train_num['Output']==train_out_col[i]].sum()
        #As denom returns a series
        denom=denom.drop(labels=['Output'])

        #Laplace smoothing (Num+1)
        Prob=(Num+1)/(denom.sum()+len(denom[denom>0]))

        #Probability of a word occuring given the output
        print("P(Text=",train_col[j],"|","Output=",train_out_col[i],")=",Prob)
        
        #Dict1 is an array of probabilities of a word given the output
        Dict1[train_col[j]+train_out_col[i]]=Prob
        
print(Dict1)

P(Text= a | Output= Sports )= 0.1111111111111111
P(Text= election | Output= Sports )= 0.05555555555555555
P(Text= fought | Output= Sports )= 0.1111111111111111
P(Text= game | Output= Sports )= 0.1111111111111111
P(Text= good | Output= Sports )= 0.1111111111111111
P(Text= it | Output= Sports )= 0.1111111111111111
P(Text= teams | Output= Sports )= 0.1111111111111111
P(Text= the | Output= Sports )= 0.1111111111111111
P(Text= was | Output= Sports )= 0.1111111111111111
P(Text= well | Output= Sports )= 0.1111111111111111
P(Text= a | Output= NS )= 0.16666666666666666
P(Text= election | Output= NS )= 0.16666666666666666
P(Text= fought | Output= NS )= 0.16666666666666666
P(Text= game | Output= NS )= 0.08333333333333333
P(Text= good | Output= NS )= 0.08333333333333333
P(Text= it | Output= NS )= 0.16666666666666666
P(Text= teams | Output= NS )= 0.08333333333333333
P(Text= the | Output= NS )= 0.08333333333333333
P(Text= was | Output= NS )= 0.16666666666666666
P(Text= well | Output= NS )= 0.1666666

In [ ]:
count=0

for k in range(0,Test.shape[0]):
    #split test sample sentence into words
    test_sentence=(Test['Text'][k]).split(" ")
    colnames=list(train_num.columns)
    print(test_sentence)

    Mul=1
    Greatest=0
    Greatest_Output='hi'

    for i in range(0,len(train_out_col)):
        Mul=1
        for j in range(0,len(test_sentence)):
            #check if word occuring in test sample sentence occurs in training sample sentences (for finding that word's probability in our dictionary )
            try:
                a=colnames.index(test_sentence[j])
            except ValueError:
                print(test_sentence[j])
                continue
            
            print("Dict:",Dict1[test_sentence[j]+train_out_col[i]])
            Mul=Dict1[test_instance[j]+train_out_col[i]]*Mul
            print(Mul)

        #P(Output|Input1,Input2,...Inputn)=P(Input1,Input2,....Inputn|Output).P(Output)
        Mul=Mul*(train_num[train_num['Output']==train_out_col[i]].shape[0])/(train_num.shape[0])
      
        #Probabilities being compared of each outcome(Sports or Not Sports) given the words of the sentence    
        if(Mul>Greatest):
            Greatest=Mul
            print(Greatest)
            Greatest_Output=train_out_col[i]
        
    print(Test['Text'][k],"is classified as:",Greatest_Output)
    if(Greatest_Output==Test['Class'][k]):
        count=count+1

#Accuracy of prediction
print("Accuracy:",(count/Test.shape[0])) 

['The', 'election', 'was']
The
Dict: 0.05555555555555555
0.05555555555555555
Dict: 0.1111111111111111
0.006172839506172839
0.004115226337448559
The
Dict: 0.16666666666666666
0.16666666666666666
Dict: 0.16666666666666666
0.027777777777777776
0.009259259259259259
The election was is classified as: NS
['What', 'a', 'match', 'it', 'was']
What
Dict: 0.1111111111111111
0.1111111111111111
match
Dict: 0.1111111111111111
0.012345679012345678
Dict: 0.1111111111111111
0.0013717421124828531
0.0009144947416552355
What
Dict: 0.16666666666666666
0.16666666666666666
match
Dict: 0.16666666666666666
0.027777777777777776
Dict: 0.16666666666666666
0.004629629629629629
0.0015432098765432098
What a match it was is classified as: NS
Accuracy: 0.5


Accuracy is less due to small training sample (50-50 split)